In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
file = './data/trajets_train.csv'
df = pd.read_csv(file,sep=",",index_col=[0]) 
traj1 = df[df['traj']==7809] #3500
print(traj1.shape)
traj2 = df[df['traj']==89117] #2500

(3487, 32)


In [3]:
new_traj1 = traj1[['timestamps_UTC', 'RS_E_InAirTemp_PC1']].copy()

# Convert 'timestamps_UTC' to datetime format if it's not already
new_traj1['timestamps_UTC'] = pd.to_datetime(new_traj1['timestamps_UTC'])

# Set 'timestamps_UTC' as the index
new_traj1.set_index('timestamps_UTC', inplace=True)


new_traj2 = traj2[['timestamps_UTC', 'RS_E_InAirTemp_PC1']].copy()

# Convert 'timestamps_UTC' to datetime format if it's not already
new_traj2['timestamps_UTC'] = pd.to_datetime(new_traj2['timestamps_UTC'])

# Set 'timestamps_UTC' as the index
new_traj2.set_index('timestamps_UTC', inplace=True)


In [5]:
print(new_traj1)

                           RS_E_InAirTemp_PC1
timestamps_UTC                               
2023-06-10 09:46:43+00:00                38.0
2023-06-10 09:47:43+00:00                37.0
2023-06-10 09:48:43+00:00                37.0
2023-06-10 09:49:43+00:00                37.0
2023-06-10 09:50:47+00:00                37.0
...                                       ...
2023-06-12 20:41:37+00:00                41.0
2023-06-12 20:42:32+00:00                47.0
2023-06-12 20:43:31+00:00                52.0
2023-06-12 20:44:32+00:00                55.0
2023-06-12 20:45:32+00:00                57.0

[3487 rows x 1 columns]


In [ ]:
traj1.set_index('timestamps_UTC', inplace=True)
traj2.set_index('timestamps_UTC', inplace=True)
# traj1 = traj1.reset_index(drop=True)
# traj2 = traj2.reset_index(drop=True)
# 
# traj1.index = pd.DatetimeIndex(traj1.index).to_period('T')
# traj2.index = pd.DatetimeIndex(traj2.index).to_period('T')

In [ ]:
# Check for duplicate timestamps in traj1
duplicate_timestamps_traj1 = traj1.index.duplicated()

# Check for duplicate timestamps in traj2
duplicate_timestamps_traj2 = traj2.index.duplicated()

# Print the results
print("Duplicate timestamps in traj1:", duplicate_timestamps_traj1.any())
print("Duplicate timestamps in traj2:", duplicate_timestamps_traj2.any())


In [ ]:
# traj1.index = pd.to_datetime(traj1.index, format='%Y-%m-%d %H:%M:%S+00:00')
# traj2.index = pd.to_datetime(traj2.index, format='%Y-%m-%d %H:%M:%S+00:00')

traj1.index = pd.DatetimeIndex(traj1.index)
traj2.index = pd.DatetimeIndex(traj2.index)
# traj1.index.asfreq('T')
# traj2.index.asfreq('T')


In [ ]:
print(traj1.index.unique())


In [7]:
# Assuming 'df' is your DataFrame with a 'timestamps' column and a 'value' column
# Adjust parameters as needed
order = (4, 1, 1)  # Example ARIMA order (p, d, q)
model = sm.tsa.ARIMA(new_traj1['RS_E_InAirTemp_PC1'], order=order)
results = model.fit()

# Make predictions
new_traj1['predictions'] = results.predict(start=new_traj1.index.min(), end=new_traj1.index.max(), dynamic=False)

# Compute residuals (difference between actual and predicted values)
new_traj1['residuals'] = new_traj1['RS_E_InAirTemp_PC1'] - new_traj1['predictions']

# Anomaly detection: Define a threshold for anomaly detection based on residuals
threshold = 2.0  # Adjust as needed
new_traj1['anomaly'] = abs(new_traj1['residuals']) > threshold

c:\Users\Vincent\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Vincent\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Vincent\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
print(traj2[traj2.index.duplicated(keep=False)])


In [ ]:
import matplotlib.pyplot as plt

# Plot real values, predicted values, and errors
plt.figure(figsize=(10, 6))

# Plot real values
plt.plot(new_traj1['timestamps_UTC'], new_traj1['RS_E_InAirTemp_PC1'], label='Real Values', color='blue')

# Plot predicted values
plt.plot(new_traj1['timestamps_UTC'], new_traj1['predictions'], label='Predicted Values', color='green')

# Plot errors
plt.plot(new_traj1['timestamps_UTC'], new_traj1['residuals'], label='Residuals', color='red')

# Highlight anomalies
anomalies = new_traj1[new_traj1['anomaly']]
plt.scatter(anomalies['timestamps_UTC'], anomalies['RS_E_InAirTemp_PC1'], color='red', label='Anomalies', marker='x')

plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.title('ARIMA Model: Real vs Predicted Values with Residuals')
plt.legend()
plt.show()
